# 1、参数详解

In [ ]:
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=4)

每次 dataloader 加载数据时，dataloader一次性创建 num_workers 个 worker，（也可以说dataloader一次性创建num_workers 个工作进程），并用batch_sampler将指定batch分配给指定worker，worker将它负责的batch加载进RAM。

然后，dataloader从RAM中找本轮迭代要用的batch，如果找到了，就使用。如果没找到，就要num_worker个worker继续加载batch到内存，直到dataloader在RAM中找到目标batch。一般情况下都是能找到的，因为batch_sampler指定batch时当然优先指定本轮要用的batch

num_worker设置得大，好处是寻batch速度快，因为下一轮迭代的batch很可能在上一轮/上上一轮…迭代时已经加载好了。坏处是内存开销大，也加重了CPU负担（worker加载数据到RAM的进程是CPU复制的）。num_workers的经验设置值是自己电脑/服务器的CPU核数，如果CPU很强、RAM也很充足，就可以设置得更大些。

如果num_worker设为0，意味着每一轮迭代时，dataloader不再有自主加载数据到RAM这一步骤（因为没有worker了），而是在RAM中找batch，找不到时再加载相应的batch。缺点当然是速度更慢。

# 2、num_worker 大小建议
Dataloader的num_worker设置多少才合适，这个问题是很难有一个推荐的值。有以下几个建议：

num_workers=0 表示只有主进程去加载 batch数据，这个可能会是一个瓶颈。

num_workers =1 示只有一个worker进程用来加载batch数据，而主进程是不参与数据加载的。这样速度也会很慢。

num_workers>0 表示指定数量的worker进程去加载数据，且主进程不参与。增加 num_works 也同时会增加 cpu内存的消耗。所以num_workers的值依赖于batch size和机器性能。

一般开始是将 num_workers 设置为等于计算机上的CPU数量

最好的办法是缓慢增加 num_workers，直到训练速度不再提高，就停止增加num_workers的值。